In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !pip install tqdm
# !pip install fancyimpute

In [ ]:
################## USED PACKAGES #######################
import pandas as pd
import numpy as np
from tqdm import tqdm
import math
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.impute import KNNImputer
from fancyimpute import IterativeImputer as MICE

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
# Loading Data - Only Train is relevant as competition closed
# Use Train for Train,Validation and Test

# Collab Directory
dirs = '/content/drive/My Drive/Colab Notebooks/COS 711 Assignment 3/data/Train.csv'
# System Directory
# dirs = './data/Train.csv'

Train = pd.read_csv(dirs)
# Used for periodic data checks
dummy_Train = pd.read_csv(dirs)

In [ ]:
# Convert Strings to Numerical Values
def replace_nan(x):
    if x==" ":
        return np.nan
    else :
        return float(x)
    
features=["temp","precip","rel_humidity","wind_dir","wind_spd","atmos_press"]
for feature in features : 
    Train[feature]=Train[feature].apply(lambda x: [ replace_nan(X) for X in x.replace("nan"," ").split(",")])
    dummy_Train[feature]=dummy_Train[feature].apply(lambda x: [ replace_nan(X) for X in x.replace("nan"," ").split(",")])

In [ ]:
# Encoding sensor location (A,B,C,D,E) as Numeric Values (0,1,2,3,4,5)
le = preprocessing.LabelEncoder()
Train['location'] = le.fit_transform(Train['location'])

In [ ]:
# type_data = ['temp', 'precip', 'rel_humidity', 'wind_dir','wind_spd', 'atmos_press']
# for k in range(len(type_data)):
#     number_of_full = []
#     for i in range(len(Train[type_data[k]])):
#         number_of_null = pd.isnull(Train[type_data[k]][i]).sum() 
#         if (121-number_of_null) == 121:
#             number_of_full.append(121-number_of_null)
#     print(len(number_of_full)/len(Train[type_data[k]]))


# not NaN's Percentage
temp_not_nans         = 0.794645730098462 * 100
precip_not_nans       = 0.815560846901345 * 100
rel_humidity_not_nans = 0.678293326468884 * 100
wind_dir_not_nans     = 0.533882489220670 * 100
wind_spd_not_nans     = 0.533882489220670 * 100
atmos_press_not_nans  = 0.820387412317395 * 100

Adding Additional Features and Removing Nan's Functions

In [ ]:
def aggregate_features(x,col_name):
    x["max_"+col_name]=x[col_name].apply(np.max)
    x["min_"+col_name]=x[col_name].apply(np.min)
    x["mean_"+col_name]=x[col_name].apply(np.mean)
    x["std_"+col_name]=x[col_name].apply(np.std)
    x["var_"+col_name]=x[col_name].apply(np.var)
    x["median_"+col_name]=x[col_name].apply(np.median)
    x["ptp_"+col_name]=x[col_name].apply(np.ptp)
    return x      
def remove_nan_values(x):
    return [e for e in x if not math.isnan(e)]

Adding Additional Features and Removing Nan's

In [ ]:
for col_name in tqdm(features):
    Train[col_name]=Train[col_name].apply(remove_nan_values)
for col_name in tqdm(features):
    Train = aggregate_features(Train,col_name)

100%|██████████| 6/6 [00:15<00:00,  2.56s/it]


## ------------------------- Mean, Median, Mode-Imputation -------------------------

In [ ]:
# Setting NaN's to the mean of the list
type_data = ['temp', 'precip', 'rel_humidity', 'wind_dir','wind_spd', 'atmos_press']
for k in range(len(type_data)):
    for i in range(len(Train[type_data[k]])):
      # Train.at[i,type_data[k]]= pd.Series(dummy_Train[type_data[k]][i]).fillna(np.nanmedian(dummy_Train[type_data[k]][i])).tolist()
      Train.at[i,type_data[k]]= pd.Series(dummy_Train[type_data[k]][i]).fillna(np.nanmean(dummy_Train[type_data[k]][i])).tolist()

In [ ]:
for x in range(121):
    Train["newtemp"+ str(x)] = Train.temp.str[x]
    Train["newprecip"+ str(x)] = Train.precip.str[x]
    Train["newrel_humidity"+ str(x)] = Train.rel_humidity.str[x]
    Train["newwind_dir"+ str(x)] = Train.wind_dir.str[x]
    Train["windspeed"+ str(x)] = Train.wind_spd.str[x]
    Train["atmospherepressure"+ str(x)] = Train.atmos_press.str[x]

In [ ]:
# Saving Cleaned and Edited DataFrame - Populating NaN's with mean values
Train.to_csv('/content/drive/My Drive/Colab Notebooks/COS 711 Assignment 3/data/data_cleaned_mean.csv')

## ------------------------- KNN-Imputation -------------------------

In [ ]:
temps = []
precips = []
rel_humiditys = []
wind_dirs = []
wind_spds = []
atmos_presss = []
for i in range(len(dummy_Train['temp'])):
    temps.append(dummy_Train['temp'][i])
    precips.append(dummy_Train['precip'][i])
    rel_humiditys.append(dummy_Train['rel_humidity'][i])
    wind_dirs.append(dummy_Train['wind_dir'][i])
    wind_spds.append(dummy_Train['wind_spd'][i])
    atmos_presss.append(dummy_Train['atmos_press'][i])
temps = np.array(temps)
precips = np.array(precips)
rel_humiditys = np.array(rel_humiditys)
wind_dirs = np.array(wind_dirs)
wind_spds = np.array(wind_spds)
atmos_presss = np.array(atmos_presss)

In [ ]:
imputer = KNNImputer(n_neighbors=10)
temps_impute = imputer.fit_transform(temps)
precips_impute = imputer.fit_transform(precips)
rel_humiditys_impute = imputer.fit_transform(rel_humiditys)
wind_dirs_impute = imputer.fit_transform(wind_dirs)
wind_spds_impute = imputer.fit_transform(wind_spds)
atmos_presss_impute = imputer.fit_transform(atmos_presss)

In [ ]:
for i in range(len(Train['temp'])):
        Train.at[i,'temp']= temps_impute[i].tolist()
        Train.at[i,'precip']= precips_impute[i].tolist()
        Train.at[i,'rel_humidity']= rel_humiditys_impute[i].tolist()
        Train.at[i,'wind_dir']= wind_dirs_impute[i].tolist()
        Train.at[i,'wind_spd']= wind_spds_impute[i].tolist()
        Train.at[i,'atmos_press']= atmos_presss_impute[i].tolist()

In [ ]:
for x in range(121):
    Train["newtemp"+ str(x)] = Train.temp.str[x]
    Train["newprecip"+ str(x)] = Train.precip.str[x]
    Train["newrel_humidity"+ str(x)] = Train.rel_humidity.str[x]
    Train["newwind_dir"+ str(x)] = Train.wind_dir.str[x]
    Train["windspeed"+ str(x)] = Train.wind_spd.str[x]
    Train["atmospherepressure"+ str(x)] = Train.atmos_press.str[x]


In [ ]:
# Saving Cleaned and Edited DataFrame - Populating NaN's with knn values
Train.to_csv('/content/drive/My Drive/Colab Notebooks/COS 711 Assignment 3/data/data_cleaned_knn.csv')

## ------------------------- MICE-Imputation -------------------------

In [ ]:
temps = []
precips = []
rel_humiditys = []
wind_dirs = []
wind_spds = []
atmos_presss = []
for i in range(len(dummy_Train['temp'])):
    temps.append(dummy_Train['temp'][i])
    precips.append(dummy_Train['precip'][i])
    rel_humiditys.append(dummy_Train['rel_humidity'][i])
    wind_dirs.append(dummy_Train['wind_dir'][i])
    wind_spds.append(dummy_Train['wind_spd'][i])
    atmos_presss.append(dummy_Train['atmos_press'][i])
temps = np.array(temps)
precips = np.array(precips)
rel_humiditys = np.array(rel_humiditys)
wind_dirs = np.array(wind_dirs)
wind_spds = np.array(wind_spds)
atmos_presss = np.array(atmos_presss)

In [ ]:
imputer = IterativeImputer(max_iter=10, random_state=1, sample_posterior=True)
temps_impute = imputer.fit_transform(temps)
precips_impute = imputer.fit_transform(precips)
rel_humiditys_impute = imputer.fit_transform(rel_humiditys)
wind_dirs_impute = imputer.fit_transform(wind_dirs)
wind_spds_impute = imputer.fit_transform(wind_spds)
atmos_presss_impute = imputer.fit_transform(atmos_presss)


In [ ]:
for i in range(len(Train['temp'])):
        Train.at[i,'temp']= temps_impute[i].tolist()
        Train.at[i,'precip']= precips_impute[i].tolist()
        Train.at[i,'rel_humidity']= rel_humiditys_impute[i].tolist()
        Train.at[i,'wind_dir']= wind_dirs_impute[i].tolist()
        Train.at[i,'wind_spd']= wind_spds_impute[i].tolist()
        Train.at[i,'atmos_press']= atmos_presss_impute[i].tolist()

In [ ]:
for x in range(121):
    Train["newtemp"+ str(x)] = Train.temp.str[x]
    Train["newprecip"+ str(x)] = Train.precip.str[x]
    Train["newrel_humidity"+ str(x)] = Train.rel_humidity.str[x]
    Train["newwind_dir"+ str(x)] = Train.wind_dir.str[x]
    Train["windspeed"+ str(x)] = Train.wind_spd.str[x]
    Train["atmospherepressure"+ str(x)] = Train.atmos_press.str[x]

In [ ]:
# Saving Cleaned and Edited DataFrame - Populating NaN's with mice values
Train.to_csv('/content/drive/My Drive/Colab Notebooks/COS 711 Assignment 3/data/data_cleaned_mice.csv')